# 2020-10-18 Exploring Why So many Big Ms.
The PTL DQN Agent gets so many Big Ms versus the Random Agent that I think something might be wrong with the logging (duplicate logging due to experience replay maybe?

In [47]:
# Common data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns;sns.set(color_codes=True)
import scipy.stats as st

# Python
import datetime
from pathlib import Path
import os

# For display
from IPython.display import display, Markdown, Latex


# To stop dataframes from compressing
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', None)

# To change resolution to match retina
%config InlineBackend.figure_format = 'retina'

#Set base dir
# Idea gotten from https://stackoverflow.com/questions/39125532/file-does-not-exist-in-jupyter-notebook
project_dir = Path(globals()['_dh'][0]+"/..").resolve()
os.chdir(project_dir)
print("Running notebook from: " + os.path.abspath(""))


from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "last_expr" # Default jupyter behavior
InteractiveShell.ast_node_interactivity = "all" # All expressions are shown.

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

plt.rc("figure", dpi=150)

#TO DO: convert this into a template.

Running notebook from: /Users/aleph/Documents/jota/tesis/ts_mcfrl
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data

In [60]:
dqn_movement_detail_report = pd.read_csv("python/data/results/dqn_few_warehouses_v4__demandgen_biased/movement_detail_report.csv")
dqn_valid_dcs = pd.read_csv("python/data/results/dqn_few_warehouses_v4__demandgen_biased/valid_dcs.csv")

custdqn_movement_detail_report = pd.read_csv("python/data/results/dqn_onehot_few_warehouses_v4/movement_detail_report.csv")

random_movement_detail_report = pd.read_csv("python/data/results/gr_random_few_warehouses/movement_detail_report.csv")
random_valid_dcs = pd.read_csv("python/data/results/gr_random_few_warehouses_debug/valid_dcs.csv")

## Comparing Big Ms in dqn

In [25]:
dqn_movement_detail_report.groupby(['episode','is_big_m']).size().reset_index().head()

,episode,is_big_m,0
0,0,False,102
1,0,True,44
2,1,False,111
3,1,True,35
4,2,False,99


No duplicates to be seen on the ep 0, so it may seem that it's the dqn that's to blame for these Big Ms. Are we looking at the same data?

In [26]:
dqn_movement_detail_report.query('episode==0').query('is_big_m').sort_values(['source_time','destination_time','destination_name']).head(5)

,source_name,destination_name,source_time,destination_time,commodity,source_kind,destination_kind,movement_type,transportation_units,transportation_cost,inventory_units,inventory_cost,customer_units,customer_cost,is_big_m,episode
8,dcs_1,c_92,5,5,34,DC,C,Delivery,0,0,0,0,18,1800000,True,0
13,dcs_2,c_27,6,6,34,DC,C,Delivery,0,0,0,0,2,200000,True,0
14,dcs_1,c_66,6,6,34,DC,C,Delivery,0,0,0,0,22,2200000,True,0
11,dcs_2,c_90,6,6,34,DC,C,Delivery,0,0,0,0,19,1900000,True,0
19,dcs_0,c_23,7,7,34,DC,C,Delivery,0,0,0,0,38,3800000,True,0


Comparing the valid movement dataset on each

In [42]:
display(dqn_valid_dcs.groupby('customer').agg(list).reset_index().sort_values('customer').head(10))
display(random_valid_dcs.groupby('customer').agg(list).reset_index().sort_values('customer').head(10))

,customer,valid_dc
0,c_10,"[dcs_0, dcs_1]"
1,c_100,"[dcs_0, dcs_2]"
2,c_101,"[dcs_0, dcs_2]"
3,c_102,"[dcs_0, dcs_2]"
4,c_11,"[dcs_0, dcs_1]"
5,c_12,"[dcs_0, dcs_1]"
6,c_13,"[dcs_0, dcs_2]"
7,c_14,"[dcs_0, dcs_2]"
8,c_15,"[dcs_1, dcs_2]"
9,c_16,"[dcs_1, dcs_2]"


,customer,valid_dc
0,c_10,"[dcs_0, dcs_1]"
1,c_100,"[dcs_0, dcs_2]"
2,c_101,"[dcs_0, dcs_2]"
3,c_102,"[dcs_0, dcs_2]"
4,c_11,"[dcs_0, dcs_1]"
5,c_12,"[dcs_0, dcs_1]"
6,c_13,"[dcs_0, dcs_2]"
7,c_14,"[dcs_0, dcs_2]"
8,c_15,"[dcs_1, dcs_2]"
9,c_16,"[dcs_1, dcs_2]"


So why does the random make better decisions in a 3 warehouse scenario environment?

Something's weird: the DQN has the 5 expected orders per step, random only has one.

In [65]:
dqn_movement_detail_report.query('episode==0').query("movement_type=='Delivery'").sort_values(['source_time','destination_time','destination_name','source_name']).head(25)
custdqn_movement_detail_report.query('episode==0').query("movement_type=='Delivery'").sort_values(['source_time','destination_time','destination_name','source_name']).head(25)
random_movement_detail_report.query('episode==0').query("movement_type=='Delivery'").sort_values(['source_time','destination_time','destination_name','source_name']).head(25)# this one doesnt match


,source_name,destination_name,source_time,destination_time,commodity,source_kind,destination_kind,movement_type,transportation_units,transportation_cost,inventory_units,inventory_cost,customer_units,customer_cost,is_big_m,episode
9,dcs_2,c_20,5,5,34,DC,C,Delivery,0,0,0,0,83,830,False,0
5,dcs_2,c_25,5,5,34,DC,C,Delivery,0,0,0,0,23,230,False,0
6,dcs_2,c_48,5,5,34,DC,C,Delivery,0,0,0,0,13,130,False,0
7,dcs_2,c_5,5,5,34,DC,C,Delivery,0,0,0,0,2,20,False,0
8,dcs_1,c_92,5,5,34,DC,C,Delivery,0,0,0,0,18,1800000,True,0
12,dcs_2,c_20,6,6,34,DC,C,Delivery,0,0,0,0,14,140,False,0
13,dcs_2,c_27,6,6,34,DC,C,Delivery,0,0,0,0,2,200000,True,0
14,dcs_1,c_66,6,6,34,DC,C,Delivery,0,0,0,0,22,2200000,True,0
10,dcs_1,c_81,6,6,34,DC,C,Delivery,0,0,0,0,25,250,False,0
11,dcs_2,c_90,6,6,34,DC,C,Delivery,0,0,0,0,19,1900000,True,0


,source_name,destination_name,source_time,destination_time,commodity,source_kind,destination_kind,movement_type,transportation_units,transportation_cost,inventory_units,inventory_cost,customer_units,customer_cost,is_big_m,episode
7,dcs_2,c_20,5,5,34,DC,C,Delivery,0,0,0,0,83,830,False,0
3,dcs_2,c_25,5,5,34,DC,C,Delivery,0,0,0,0,23,230,False,0
4,dcs_1,c_48,5,5,34,DC,C,Delivery,0,0,0,0,13,130,False,0
5,dcs_2,c_5,5,5,34,DC,C,Delivery,0,0,0,0,2,20,False,0
6,dcs_0,c_92,5,5,34,DC,C,Delivery,0,0,0,0,18,180,False,0
10,dcs_1,c_20,6,6,34,DC,C,Delivery,0,0,0,0,14,140,False,0
11,dcs_0,c_27,6,6,34,DC,C,Delivery,0,0,0,0,2,20,False,0
12,dcs_1,c_66,6,6,34,DC,C,Delivery,0,0,0,0,22,2200000,True,0
8,dcs_2,c_81,6,6,34,DC,C,Delivery,0,0,0,0,25,2500000,True,0
9,dcs_1,c_90,6,6,34,DC,C,Delivery,0,0,0,0,19,190,False,0


,source_name,destination_name,source_time,destination_time,commodity,source_kind,destination_kind,movement_type,transportation_units,transportation_cost,inventory_units,inventory_cost,customer_units,customer_cost,is_big_m,episode
4,dcs_1,c_32,5,5,34,DC,C,Delivery,0,0,0,0,18,180,False,0
8,dcs_2,c_40,5,5,34,DC,C,Delivery,0,0,0,0,11,1100000,True,0
6,dcs_0,c_42,5,5,34,DC,C,Delivery,0,0,0,0,2,20,False,0
7,dcs_1,c_7,5,5,34,DC,C,Delivery,0,0,0,0,8,80,False,0
5,dcs_1,c_96,5,5,34,DC,C,Delivery,0,0,0,0,22,220,False,0
13,dcs_1,c_20,6,6,34,DC,C,Delivery,0,0,0,0,83,830,False,0
9,dcs_1,c_25,6,6,34,DC,C,Delivery,0,0,0,0,23,230,False,0
10,dcs_1,c_48,6,6,34,DC,C,Delivery,0,0,0,0,13,130,False,0
11,dcs_1,c_5,6,6,34,DC,C,Delivery,0,0,0,0,2,200000,True,0
12,dcs_1,c_92,6,6,34,DC,C,Delivery,0,0,0,0,18,1800000,True,0
